## Определение целевой переменной **RainTomorrow** 

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
import time
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/vadim0912/MLIntro2022_Spring/main/lecture03/HW2/weather.csv')


In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

In [4]:
ohe = OneHotEncoder(sparse=False)

In [5]:
data = data.drop("Unnamed: 0", axis=1)
data = data.drop("Cloud9am", axis=1)
data = data.drop("Cloud3pm", axis=1)
data = data.drop("Evaporation", axis=1)
data = data.drop("Sunshine", axis=1)
data["Date"] = data["Date"].apply(lambda x: int(x.replace("-", "")))

data["Location"] = ohe.fit_transform(data[["Location"]])
# data = pd.get_dummies(data, columns=["Location", "WindGustDir", "WindDir9am", "WindDir3pm"])

data["MinTemp"] = data["MinTemp"].fillna(data["MinTemp"].mean())
data["MaxTemp"] = data["MaxTemp"].fillna(data["MaxTemp"].mean())
data["Rainfall"] = data["Rainfall"].fillna(data["Rainfall"].mean())

data["WindGustSpeed"] = data["WindGustSpeed"].fillna(data["WindGustSpeed"].mean())
data["WindSpeed9am"] = data["WindSpeed9am"].fillna(data["WindSpeed9am"].mean())
data["WindSpeed3pm"] = data["WindSpeed3pm"].fillna(data["WindSpeed3pm"].mean())

data["WindGustDir"] = ohe.fit_transform(data[["WindGustDir"]])
data["WindDir3pm"] = ohe.fit_transform(data[["WindDir3pm"]])
data["WindDir9am"] = ohe.fit_transform(data[["WindDir9am"]])

data["Humidity9am"] = data["Humidity9am"].fillna(data["Humidity9am"].mean())
data["Humidity3pm"] = data["Humidity3pm"].fillna(data["Humidity3pm"].mean())
data["Pressure9am"] = data["Pressure9am"].fillna(data["Pressure9am"].mean())
data["Pressure3pm"] = data["Pressure3pm"].fillna(data["Pressure3pm"].mean())

data["Temp9am"] = data["Temp9am"].fillna(data["Temp9am"].mean())
data["Temp3pm"] = data["Temp3pm"].fillna(data["Temp3pm"].mean())
data["RainToday"] = data["RainToday"].replace({"No": 0, "Yes":1})
data["RainToday"] = data["RainToday"].fillna(0)  # remake 
data["RainTomorrow"] = data["RainTomorrow"].replace({"No": 0, "Yes":1})

In [6]:
ohe_features_location = ohe.fit_transform(data[["Location"]])

ohe_features_MinTemp = data[["MinTemp"]]
ohe_features_MaxTemp = data[["MaxTemp"]]
ohe_features_Rainfall = data[["Rainfall"]]

ohe_features_WindGustSpeed = data[["WindGustSpeed"]]
ohe_features_WindSpeed9am = data[["WindSpeed9am"]]
ohe_features_WindSpeed3pm = data[["WindSpeed3pm"]]

ohe_features_WindGustDir = ohe.fit_transform(data[["WindGustDir"]])
ohe_features_WindDir3pm = ohe.fit_transform(data[["WindDir3pm"]])
ohe_features_WindDir9am = ohe.fit_transform(data[["WindDir9am"]])

ohe_features_Humidity9am = data[["Humidity9am"]]
ohe_features_Humidity3pm = data[["Humidity3pm"]]
ohe_features_Pressure9am = data[["Pressure9am"]]
ohe_features_Pressure3pm = data[["Pressure3pm"]]
ohe_features_Temp9am = data[["Temp9am"]]
ohe_features_Temp3pm = data[["Temp3pm"]]
ohe_features_RainToday = data[["RainToday"]]

X_encoded = np.hstack((ohe_features_location, ohe_features_MinTemp.values,
                       ohe_features_MaxTemp.values, ohe_features_Rainfall.values,
                       ohe_features_WindGustSpeed.values, ohe_features_WindSpeed9am.values,
                       ohe_features_WindSpeed3pm.values, ohe_features_WindGustDir,
                       ohe_features_WindDir9am, ohe_features_WindDir3pm,
                       ohe_features_Humidity9am.values, ohe_features_Humidity3pm.values,
                       ohe_features_Pressure9am.values, ohe_features_Pressure3pm.values,
                       ohe_features_Temp9am.values, ohe_features_Temp3pm.values,
                       ohe_features_RainToday.values))
y = data["RainTomorrow"]
X_encoded.shape

(142193, 21)

Choose desired scaler

In [7]:
mm_scaler = MinMaxScaler()
X_encoded = mm_scaler.fit_transform(X_encoded)

In [8]:
rs_scaler = RobustScaler()
X_encoded = rs_scaler.fit_transform(X_encoded)

In [9]:
ss_scaler = StandardScaler()
X_encoded = ss_scaler.fit_transform(X_encoded)

In [10]:
n_scaler = Normalizer(norm='l2')
X_encoded = n_scaler.fit_transform(X_encoded)

# Логистическая регрессия
sklearn.linear_model.LogisticRegression

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.25, shuffle=False)

In [12]:
# %%timeit
lr = LogisticRegression()
lr.fit(X_train, y_train)
test_predicted = lr.predict(X_test)

In [13]:
print(classification_report(y_test, test_predicted))

              precision    recall  f1-score   support

           0       0.87      0.96      0.91     27882
           1       0.74      0.46      0.57      7667

    accuracy                           0.85     35549
   macro avg       0.80      0.71      0.74     35549
weighted avg       0.84      0.85      0.83     35549



# Метод К-близжайших соседей

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [15]:
# %%timeit
knc = KNeighborsClassifier(n_neighbors=16)
knc.fit(X_train, y_train)
test_predicted = knc.predict(X_test)

In [16]:
print(classification_report(y_test, test_predicted))

              precision    recall  f1-score   support

           0       0.86      0.96      0.91     27882
           1       0.74      0.43      0.54      7667

    accuracy                           0.84     35549
   macro avg       0.80      0.69      0.73     35549
weighted avg       0.83      0.84      0.83     35549



Commended code helps us to see, that n_neighbors=16 is optimal value

In [20]:
# val_scores = list()
# for k in np.arange(1, 17, 5):
#     knc = KNeighborsClassifier(n_neighbors=k)
#     knc.fit(X_train, y_train)
#     test_predicted = knc.predict(X_test)
#     acc_score = accuracy_score(test_predicted, y_test)

#     val_scores.append(acc_score)

# plt.plot(np.arange(1, 17, 5), val_scores)
# plt.xticks(np.arange(1, 17, 5))
# plt.xlabel("Neibours number")
# plt.ylabel("accuracy_score")
# plt.show()

# Байесовский классификатор

In [21]:
from sklearn.naive_bayes import GaussianNB

In [22]:
# %%timeit
gnb = GaussianNB()
gnb.fit(X_train, y_train)
test_predicted = gnb.predict(X_test)

In [23]:
print(classification_report(y_test, test_predicted))

              precision    recall  f1-score   support

           0       0.89      0.86      0.88     27882
           1       0.55      0.63      0.59      7667

    accuracy                           0.81     35549
   macro avg       0.72      0.74      0.73     35549
weighted avg       0.82      0.81      0.81     35549



# Вывод
В дз было необходимо сперва провести предобработку данных, что сильно сказывается на построенних и результатах будущей модели.
После преобработки  применял поочередно различные методы масштабирования данных (различия в рузьтатах незначительны), где MinMaxScaler проявил себя лучше остальных.
Время выполнения наименьшее у логистической регрессии, но результаты хуже, чем у байесовского классификатора. Методо близжайших соседей и самый медленный, и наименее точный.